In [18]:
using DataFrames
using DelimitedFiles
using SparseArrays
using LinearAlgebra
using MatrixMarket

In [55]:
function find_lines_with_character(file_path::AbstractString, character::AbstractString)
    matching_lines = Vector{Int}()

    # Open the file
    file = open(file_path, "r")

    # Read the file line by line and search for the character
    line_number = 0
    for line in eachline(file)
        line_number += 1

        if contains(line, character)
            push!(matching_lines, line_number)
        end
    end

    close(file)

    return matching_lines
end



# Example usage:
file_name = "QPLIB_0018"
file_path = "QPLIB_0018\\QPLIB_0018.qplib"
character_to_search = "#"

line_list = find_lines_with_character(file_path, character_to_search);


In [56]:
Lines = readlines(file_path)

num_var = Int(parse(Float64,split(Lines[line_list[1]])[1]))
num_const = Int(parse(Float64,split(Lines[line_list[2]])[1]))
num_quadterms = Int(parse(Float64,split(Lines[line_list[3]])[1]))

defval_lincoeff = parse(Float64,split(Lines[line_list[4]])[1])
numndef_lincoeff = parse(Float64,split(Lines[line_list[5]])[1])

obj_const = parse(Float64,split(Lines[line_list[6]])[1])

numlin_const = parse(Float64,split(Lines[line_list[7]])[1])

val_inf = Float64(parse(BigFloat,split(Lines[line_list[8]])[1]))

def_lhs_val = parse(Float64,split(Lines[line_list[9]])[1])
numndef_lhs = parse(Float64,split(Lines[line_list[10]])[1])

def_rhs_val = parse(Float64,split(Lines[line_list[11]])[1])
numndef_rhs = parse(Float64,split(Lines[line_list[12]])[1])

def_var_lb = parse(Float64,split(Lines[line_list[13]])[1])
numndef_var_lb = parse(Float64,split(Lines[line_list[14]])[1])

def_var_ub = Float64(parse(BigFloat,split(Lines[line_list[15]])[1]))
numndef_var_ub = parse(Float64,split(Lines[line_list[16]])[1])

def_var_prime = Float64(parse(Float64,split(Lines[line_list[17]])[1]))
numndef_var_prime = parse(Float64,split(Lines[line_list[18]])[1])

def_const_dual = Float64(parse(Float64,split(Lines[line_list[19]])[1]))
numndef_const_dual = parse(Float64,split(Lines[line_list[20]])[1])

def_var_bdual = Float64(parse(Float64,split(Lines[line_list[21]])[1]))
numndef_var_bdual = parse(Float64,split(Lines[line_list[22]])[1])

numndef_var_name = parse(Float64,split(Lines[line_list[23]])[1])
numndef_const_name = parse(Float64,split(Lines[line_list[24]])[1])


0.0

In [57]:
# Hessian Extractor ✓
# Linear Constraints 
# Lhs and Rhs Values
# lb and ub values
# prime, constraint dual, variable bound dual

b_length = (line_list[4]-1)-(line_list[3])
B_cord = zeros(b_length,3)
for i in 1:b_length
    B_cord[i,:] = parse.(Float64,split(Lines[line_list[3] + i])[1:3])
end
B_sparse = sparse(B_cord[:,1],B_cord[:,2],B_cord[:,3])

B = mmwrite(string(file_name, "_B", ".mtx"),B_sparse)